In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')
from fbprophet.plot import plot_plotly
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import *
from fbprophet import Prophet
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("../input/bike-sharing-dataset/day.csv")

In [ ]:
df.head()

In [ ]:
df = df[['dteday', 'cnt', 'temp']].dropna()

In [ ]:
df['date_index'] = df['dteday']

In [ ]:
df['date_index'] = pd.to_datetime(df['date_index'])
df = df.set_index('date_index')

In [ ]:
df.columns=['ds','y','temp']

In [ ]:
df.head()

In [ ]:
plt.figure(figsize = (16, 7))
plt.plot(df.index,df.y)
plt.show()

In [ ]:
m=Prophet(interval_width=0.95,yearly_seasonality=True)
m.fit(df)

In [ ]:
future=m.make_future_dataframe(periods=50,include_history=True)

In [ ]:
forecast=m.predict(future)
preds= forecast['yhat'][:-50]

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(df.index,preds,color='red',label=' Predictions')
plt.plot(df.index,df.y,color='blue',label='Actual')
plt.legend()
plt.show()

In [ ]:
r2 = round(r2_score(df["y"], preds), 3)
mse = round(mean_squared_error(df["y"], preds), 3)
mae = round(mean_absolute_error(df["y"], preds), 3)
print("R2: ", r2)
print("MSE: ", mse)
print("MAE: ", mae)

<h2>Adding Regressor</h2>

In [ ]:
future_range = pd.date_range('2013-01-01', periods=10, freq='D')
future_temp_df = pd.DataFrame({ 'future_date': future_range, 'future_temp' : 0})
future_temp_df['future_date'] = pd.to_datetime(future_temp_df['future_date'])
future_temp_df = future_temp_df.set_index('future_date')

In [ ]:
future_temp_df.at['2013-01-01', 'future_temp'] = 0.319148
future_temp_df.at['2013-01-02', 'future_temp'] = 0.255319
future_temp_df.at['2013-01-03', 'future_temp'] = 0.234042
future_temp_df.at['2013-01-04', 'future_temp'] = 0.319148
future_temp_df.at['2013-01-05', 'future_temp'] = 0.340425
future_temp_df.at['2013-01-06', 'future_temp'] = 0.404255
future_temp_df.at['2013-01-07', 'future_temp'] = 0.361702
future_temp_df.at['2013-01-08', 'future_temp'] = 0.404255
future_temp_df.at['2013-01-09', 'future_temp'] = 0.425531
future_temp_df.at['2013-01-10', 'future_temp'] = 0.446808

In [ ]:
future_temp_df.tail(10)

In [ ]:
def weather_temp(ds):
    date = (pd.to_datetime(ds)).date()  
    if df[date:].empty:
        return future_temp_df[date:]['future_temp'].values[0]
    else:
        return (df[date:]['temp']).values[0]    
    return 0

In [ ]:
df.columns=['ds','y','temp']
m = Prophet()
m.add_regressor('temp')
m.fit(df)

In [ ]:
future = m.make_future_dataframe(periods=10)
future['temp'] = future['ds'].apply(weather_temp)

In [ ]:
forecast=m.predict(future)

In [ ]:
preds= forecast['yhat'][:-10]

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(df.index,preds,color='red',label=' Predictions')
plt.plot(df.index,df.y,color='blue',label='Actual')
plt.legend()
plt.show()

In [ ]:
r2 = round(r2_score(df["y"], preds), 3)
mse = round(mean_squared_error(df["y"], preds), 3)
mae = round(mean_absolute_error(df["y"], preds), 3)
print("R2: ", r2)
print("MSE: ", mse)
print("MAE: ", mae)

In [ ]:
forecast=m.predict(future)
preds= forecast['yhat']
figure=m.plot(forecast,xlabel='Date',ylabel='Price')
figure.set_size_inches(15,6)

<h2>Adding Multiple Regressor<\h2>

In [ ]:
df=pd.read_csv("../input/bike-sharing-dataset/day.csv")
df = df[['dteday', 'cnt', 'temp','weathersit']].dropna()

In [ ]:
df['date_index'] = df['dteday']
df['date_index'] = pd.to_datetime(df['date_index'])
df = df.set_index('date_index')


In [ ]:
df.columns=['ds','y','temp','weathersit']

In [ ]:
df.head()

In [ ]:
def weather_temp(ds):
    date = (pd.to_datetime(ds)).date()
    
    if df[date:].empty:
        return future_temp_df[date:]['future_temp'].values[0]
    else:
        return (df[date:]['temp']).values[0]
    
    return 0

def weather_condition(ds):
    date = (pd.to_datetime(ds)).date()
    
    if df[date:].empty:
        return future_temp_df[date:]['future_weathersit'].values[0]
    else:
        return (df[date:]['weathersit']).values[0]
    
    return 0

In [ ]:
future_range = pd.date_range('2013-01-01', periods=10, freq='D')
future_temp_df = pd.DataFrame({ 'future_date': future_range, 'future_temp' : 0, 'future_weathersit': 0})

future_temp_df['future_date'] = pd.to_datetime(future_temp_df['future_date'])
future_temp_df = future_temp_df.set_index('future_date')

future_temp_df.at['2013-01-01', 'future_temp'] = 0.319148
future_temp_df.at['2013-01-02', 'future_temp'] = 0.255319
future_temp_df.at['2013-01-03', 'future_temp'] = 0.234042
future_temp_df.at['2013-01-04', 'future_temp'] = 0.319148
future_temp_df.at['2013-01-05', 'future_temp'] = 0.340425
future_temp_df.at['2013-01-06', 'future_temp'] = 0.404255
future_temp_df.at['2013-01-07', 'future_temp'] = 0.361702
future_temp_df.at['2013-01-08', 'future_temp'] = 0.404255
future_temp_df.at['2013-01-09', 'future_temp'] = 0.425531
future_temp_df.at['2013-01-10', 'future_temp'] = 0.446808

future_temp_df.at['2013-01-01', 'future_weathersit'] = 4
future_temp_df.at['2013-01-02', 'future_weathersit'] = 4
future_temp_df.at['2013-01-03', 'future_weathersit'] = 4
future_temp_df.at['2013-01-04', 'future_weathersit'] = 4
future_temp_df.at['2013-01-05', 'future_weathersit'] = 4
future_temp_df.at['2013-01-06', 'future_weathersit'] = 4
future_temp_df.at['2013-01-07', 'future_weathersit'] = 4
future_temp_df.at['2013-01-08', 'future_weathersit'] = 4
future_temp_df.at['2013-01-09', 'future_weathersit'] = 4
future_temp_df.at['2013-01-10', 'future_weathersit'] = 4

In [ ]:
m = Prophet()
m.add_regressor('temp')
m.add_regressor('weathersit')
m.fit(df)

In [ ]:
future = m.make_future_dataframe(periods=10)
future['temp'] = future['ds'].apply(weather_temp)
future['weathersit'] = future['ds'].apply(weather_condition)

In [ ]:
forecast=m.predict(future)
preds= forecast['yhat'][:-10]

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(df.index,preds,color='red',label=' Predictions')
plt.plot(df.index,df.y,color='blue',label='Actual')
plt.legend()
plt.show()

In [ ]:
r2 = round(r2_score(df["y"], preds), 3)
mse = round(mean_squared_error(df["y"], preds), 3)
mae = round(mean_absolute_error(df["y"], preds), 3)
print("R2: ", r2)
print("MSE: ", mse)
print("MAE: ", mae)

In [ ]:
forecast=m.predict(future)
preds= forecast['yhat']
figure=m.plot(forecast,xlabel='Date',ylabel='Price')
figure.set_size_inches(15,6)